In [62]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
import torch.nn.functional as F
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from sklearn.metrics import mean_absolute_error
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

In [204]:
gid = 0
srv = 8
c = 0
k = 0
dataset = 3
test_error = {}
for fold_num in range(5):
    test_error[fold_num] = {}
    for lr in [0.001, 0.01, 0.1]:
        test_error[fold_num][lr] = {}
        for iters in [20000]:
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'], 3)):
                for p in [0.0]:
                    
                    try:
                        o = "\', \'".join(str(x) for x in order)

                        directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/{}/".format(dataset, fold_num, lr, iters, p)
                        filename = "test-error-[\'{}\'].npy".format(o)
                        full_path = directory + filename
                        my_file = Path(full_path)
    #                     print(full_path)
                        e = np.load(full_path).item()
                        test_error[fold_num][lr][order] = {}
                        for iters in range(1000, 20001, 1000):
                            test_error[fold_num][lr][order][iters] = e[iters]
                    except:
#                         print("except")
                        continue

In [205]:
appliance_contri = {'hvac':0.83003428, 'fridge':0.0827564, 'dr':0.06381463, 'dw':0.01472098, 'mw':0.00867371}

In [206]:
error = {}

for fold_num in range(5):
    error[fold_num] = {}
    for lr in [0.001 ,0.01, 0.1]:
        error[fold_num][lr] = {}
        for order in test_error[fold_num][lr].keys():
            error[fold_num][lr][order] = {}
            for iters in range(1000, 20001, 1000):
                apps = test_error[fold_num][lr][order][iters].keys()
                error[fold_num][lr][order][iters] = 0
                for k in apps:
                    error[fold_num][lr][order][iters] += test_error[fold_num][lr][order][iters][k]/appliance_contri[k]

In [207]:
b_error = {}
for fold_num in range(5):
    b_error[fold_num] = {}
    for lr in [0.001 ,0.01, 0.1]:
        b_error[fold_num][lr] = {}
        for order in test_error[fold_num][lr].keys():
#             b_error[fold_num][lr][order] = {}
#             for iters in range(1000, 100001, 1000):
            b_error[fold_num][lr][order] = pd.Series(error[fold_num][lr][order]).min()

In [210]:
pd.DataFrame(b_error[3]).T.min().nsmallest(3)

mw      fridge  hvac    1484.358404
fridge  hvac    mw      1524.857902
hvac    fridge  mw      1533.993993
dtype: float64

In [211]:
cand = {}
for fold_num in range(5):
    c = pd.DataFrame(b_error[fold_num]).T.min().nsmallest(3).keys()
    cand[fold_num] = []
    for o in c:
        k = [x for x in APPLIANCE_ORDER[1:-1] if x not in o]
        a = o + (k[0],)
        cand[fold_num].append(a)

In [222]:
cand[4]

[('dw', 'fridge', 'hvac', 'dr'),
 ('fridge', 'hvac', 'dw', 'dr'),
 ('hvac', 'fridge', 'dw', 'dr')]

In [213]:
gid = 0
srv = 8
c = 0
k = 0
dataset = 3
test_error = {}
for fold_num in range(5):
    test_error[fold_num] = {}
    for lr in [0.01, 0.001, 0.1]:
        test_error[fold_num][lr] = {}
        for iters in [20000]:
            for order in cand[fold_num]:
                for p in [0.0]:
                    
                    try:
                        o = "\', \'".join(str(x) for x in order)
                        
                        directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/{}/".format(dataset, fold_num, lr, iters, p)
                        filename = "test-error-[\'{}\'].npy".format(o)
#                         print(filename)
                        full_path = directory + filename
                        my_file = Path(full_path)
#                         print(full_path)
                        e = np.load(full_path).item()
                        test_error[fold_num][lr][order] = {}
                        for iters in range(1000, 20001, 1000):
                            test_error[fold_num][lr][order][iters] = e[iters]
                    except:
                        o = " ".join(str(x) for x in order)
                        line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 3 {} {} {} {} {} &".format(gid, lr, 20000, p, fold_num, o))
                        print(line)
                        gid += 1
                        if gid == 4:
                            gid = 0

CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 3 0.01 20000 0.0 0 dw hvac fridge dr &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 3 0.01 20000 0.0 0 fridge hvac dw dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 3 0.01 20000 0.0 0 hvac fridge dw dr &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 3 0.001 20000 0.0 0 dw hvac fridge dr &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 3 0.001 20000 0.0 0 fridge hvac dw dr &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 3 0.001 20000 0.0 0 hvac fridge dw dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 3 0.1 20000 0.0 0 dw hvac fridge dr &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 3 0.1 20000 0.0 0 fridge hvac dw dr &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 3 0.1 20000 0.0 0 hvac fridge dw dr &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 3 0.01 20000 0.0 1 dr hvac fridge dw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 3 0.001 20000 0.0 1 dr hvac fridge dw &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 3 0.1 20000 0.0 1 dr hvac fridge dw &
CUDA_VISIBLE_DEVICES=0 pytho

In [214]:
error = {}

for fold_num in range(5):
    error[fold_num] = {}
    for lr in [0.01]:
        error[fold_num][lr] = {}
        for order in test_error[fold_num][lr].keys():
            error[fold_num][lr][order] = {}
            for iters in range(1000, 20001, 1000):
                apps = test_error[fold_num][lr][order][iters].keys()
                error[fold_num][lr][order][iters] = 0
                for k in apps:
                    error[fold_num][lr][order][iters] += test_error[fold_num][lr][order][iters][k]/appliance_contri[k]

In [215]:
b_error = {}
iters = {}
for fold_num in range(5):
    b_error[fold_num] = {}
    iters[fold_num] = {}
    for lr in [0.01]:
        b_error[fold_num][lr] = {}
        iters[fold_num][lr] = {}
        for order in test_error[fold_num][lr].keys():
#             b_error[fold_num][lr][order] = {}
#             for iters in range(1000, 100001, 1000):
            b_error[fold_num][lr][order] = pd.Series(error[fold_num][lr][order]).min()
            iters[fold_num][lr][order] = pd.Series(error[fold_num][lr][order]).argmin()
        

/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  del sys.path[0]


In [216]:
pd.DataFrame(b_error[2])

,,,,0.01
mw,fridge,hvac,dr,2730.886539


In [217]:
pd.DataFrame(iters[0])

,0.01


In [225]:
cnn_tree_params = {}
cnn_tree_params[1] = np.load("../code/baseline/result/cnn-tree-new-param-1.npy").item()
cnn_tree_params[3] = np.load("../code/baseline/result/cnn-tree-final-param-3.npy").item()

In [232]:
# get prediction for cnn tree
cnn_tree_error= {}
for dataset in [1, 3]:
    cnn_tree_error[dataset] = {}
    for fold_num in range(5):
#         cnn_tree_error[dataset][fold_num] = {}

        lr = cnn_tree_params[dataset][fold_num]['lr']
        iters = cnn_tree_params[dataset][fold_num]['iters']
        order = cnn_tree_params[dataset][fold_num]['order']

        o = "\', \'".join(str(x) for x in order)
        directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num,lr)
        filename = "test-error-[\'{}\'].npy".format(o)

        full_path = directory + filename
        cnn_tree_error[dataset][fold_num] = np.load(full_path).item()[iters]

#         for idx, appliance in enumerate(order):
#             cnn_tree_error[dataset][fold_num][appliance] = cnn_pred[idx]
    

In [240]:
error = {}
error[1] = {}
error[1][0] = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['dr', 'hvac', 'fridge', 'mw', 'dw'].npy").item()
error[1][1] = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['hvac', 'dr', 'fridge', 'dw', 'mw'].npy").item()
error[1][2] = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['fridge', 'hvac', 'mw', 'dw', 'dr'].npy").item()
error[1][3] = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['hvac', 'mw', 'fridge', 'dw', 'dr'].npy").item()
error[1][4] = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['fridge', 'hvac', 'dw', 'mw', 'dr'].npy").item()

In [249]:
k = pd.DataFrame(error[1][0]).mean().min()*14 + pd.DataFrame(error[1][1]).mean().min()*14 + pd.DataFrame(error[1][2]).mean().min()*14 + pd.DataFrame(error[1][3]).mean().min()*13 + pd.DataFrame(error[1][4]).mean().min()*13

In [250]:
k/68

62.03123642656481

In [257]:
error = {}
error[1] = {}
error[1][0] = np.load("../code/baseline/cnn-tree/3/0/0.01/20000/0.0/test-error-['dw', 'hvac', 'fridge', 'dr', 'mw'].npy").item()
error[1][1] = np.load("../code/baseline/cnn-tree/3/0/0.01/20000/0.0/test-error-['fridge', 'hvac', 'dr', 'dw', 'mw'].npy").item()
error[1][2] = np.load("../code/baseline/cnn-tree/3/0/0.01/20000/0.0/test-error-['mw', 'fridge', 'hvac', 'dr', 'dw'].npy").item()
error[1][3] = np.load("../code/baseline/cnn-tree/3/0/0.01/20000/0.0/test-error-['hvac', 'fridge', 'mw', 'dr', 'dw'].npy").item()
error[1][4] = np.load("../code/baseline/cnn-tree/3/0/0.01/20000/0.0/test-error-['fridge', 'hvac', 'dw', 'dr', 'mw'].npy").item()

In [258]:
k = pd.DataFrame(error[1][0]).mean().min()*14 + pd.DataFrame(error[1][1]).mean().min()*14 + pd.DataFrame(error[1][2]).mean().min()*14 + pd.DataFrame(error[1][3]).mean().min()*13 + pd.DataFrame(error[1][4]).mean().min()*13

In [264]:
pd.DataFrame(error[1][1]).mean().min()

77.416244615561567